In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 23.2 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/avisinghal6/VIDEO_CAPTIONING_PIPELINE.git

Cloning into 'VIDEO_CAPTIONING_PIPELINE'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 45 (delta 20), reused 24 (delta 8), pack-reused 0
Unpacking objects: 100% (45/45), 8.52 MiB | 8.69 MiB/s, done.


In [ ]:
%cd VIDEO_CAPTIONING_PIPELINE

/content/VIDEO_CAPTIONING_PIPELINE


In [ ]:
!pip install --upgrade yt-dlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.2/357.2 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 22.3 MB/s eta 0:00:00


In [ ]:
!python download_youcookii_videos.py

[generic] Extracting URL: www.youtube.com/watch?v=fn9anlEL4FI
[youtube] Extracting URL: http://www.youtube.com/watch?v=fn9anlEL4FI
[youtube] fn9anlEL4FI: Downloading webpage
[youtube] fn9anlEL4FI: Downloading android player API JSON
[info] fn9anlEL4FI: Downloading 1 format(s): 247+251
[dashsegments] Total fragments: 9
[download] Destination: /content/VIDEO_CAPTIONING_PIPELINE/fn9anlEL4FI.f247.webm
[download] 100% of   82.54MiB in 00:00:13 at 5.92MiB/s
[dashsegments] Total fragments: 1
[download] Destination: /content/VIDEO_CAPTIONING_PIPELINE/fn9anlEL4FI.f251.webm
[download] 100% of    6.93MiB in 00:00:00 at 7.81MiB/s
[Merger] Merging formats into "/content/VIDEO_CAPTIONING_PIPELINE/fn9anlEL4FI.webm"
Deleting original file /content/VIDEO_CAPTIONING_PIPELINE/fn9anlEL4FI.f247.webm (pass -k to keep)
Deleting original file /content/VIDEO_CAPTIONING_PIPELINE/fn9anlEL4FI.f251.webm (pass -k to keep)
[INFO] Downloaded video fn9anlEL4FI
[generic] Extracting URL: www.youtube.com/watch?v=-dh_uGah

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image
from frames import *
from model import *
from captions import *
from yolo import *
import cv2

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


In [ ]:
#GENERATING FRAMES
videos_list={}
videos_time_stamps=get_saving_frames_durations(5,"dvc_results.json")
for time_stamp in videos_time_stamps:
  for key,value in time_stamp.items():
    generate_frames(f'{key}.webm',value,videos_list)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#GENERATING FEATURES
img2vec = Img2VecResnet18(device) 
videos_features_vectors={}
for time_stamp in videos_time_stamps:
  allVectors={}
  for key,value in time_stamp.items():
    for image in tqdm(os.listdir(f'{key}-opencv')):
      I = Image.open(os.path.join(f'{key}-opencv', image))
      vec = img2vec.getVec(I)
      allVectors[image] = vec
      I.close() 
    videos_features_vectors[key]=allVectors

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 321MB/s]
100%|██████████| 648/648 [01:06<00:00,  9.78it/s]


In [ ]:
#COMPUTING SIMILARITY BETWEEN FRAMES
from collections import defaultdict
result=get_similar_frames(videos_features_vectors,videos_list)

In [ ]:
#GETTING FINAL FRAMES FOR EACH VIDEO
final_frames=get_final_frames(result)

In [ ]:
#USE FINAL FRAMES FOR CAPTIONING

In [ ]:
#USING YOLO FOR OBJECT DETECTION AND ADDING THE BOUNDING BOXES ALONG WITH LABELS
for k,v in final_frames.items():
  for k1,v1 in v.items():
    yolo_box(k,v1)


In [ ]:
#GET CAPTIONS
config={
    "processor": "Salesforce/blip-image-captioning-base",

    "model":"Salesforce/blip-image-captioning-base"
}
processor, model = get_model(config,device)
captions = get_captions(final_frames, processor, model,device)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
captions


{'ysRLGUndzgg': 'a woman is making a bowl of food. a person is putting food into a bowl. a person is cooking food on a stove. a person is putting a bowl with food. a woman is preparing food in the kitchen. ',
 'Mzn6Q4gUDBo': 'two women in a kitchen with a bowl of food. a person is peeling a piece of paper. a person stirring food into a pot. a person is stirring something into a pot. a blury background with the words jane looking 1. a pot of boiling on a stove. ',
 'JylDlRtH9Tc': 'a woman standing in a kitchen preparing food. a person is putting some food into a bowl. ',
 'pxQd53yvSaA': 'a person is adding rice into a container. a frying pan with some food in it. a piece of paper. ',
 'HF49t8uVJOE': 'a woman in an apron is making food. ',
 'fn9anlEL4FI': 'how to cook corn corn corn corn corn corn corn corn corn corn corn corn corn corn corn corn. how to cook a curry. ',
 'vVZsj1t9R70': 'a person cutting up some food on a cutting board. a pot of soup with a spoon in it. a pot filled with

In [ ]:
#ADD CODE FOR LLM